In [ ]:
import numpy as np
import pandas as pd
import gensim
from gensim.models import Word2Vec, KeyedVectors
from gensim.models import FastText
from tqdm import tqdm

In [ ]:
#中文維基詞向量-路徑
zhwiki_path = r'D:\Code\NTUT_Thesis\model_data\zhwiki\wiki.zh.vector'
#fastText Facebook 20萬字訓練詞向量-路徑
fasttext_path = r'D:\Code\NTUT_Thesis\model_data\fastText_cbow_300d_facebook\cc.zh.300.vec'

In [ ]:
# 載入詞向量function
def load_fasttext():
        print('loading word embeddings...')
        embeddings_index = {}
        f = open(r'D:\Code\NTUT_Thesis\model_data\fastText_cbow_300d_facebook\cc.zh.300.vec', encoding='utf-8')
        for line in tqdm(f):
                values = line.strip().rsplit(' ')
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
        f.close()
        print('found %s word vectors' % len(embeddings_index))
        return embeddings_index

In [ ]:
# 載入詞向量
model = load_fasttext()
#model的資料結構是dictionary

In [ ]:
vec = model['視障']
print(vec.shape)
print(vec)

In [ ]:
# 若未知詞使用『全零向量』填充，這邊製作全零向量
vec_size = next(iter(model.items()))[1]
unknown = np.zeros((vec_size.shape), dtype=np.float32)
print(unknown.shape)
print(unknown)

In [ ]:
#好...我們現在有一組keyword存在List裡面
#['視障', '愛盲', '林佳臻', '基金會', '媽媽', '家庭', '成為', '看見', '今年', '母親', '表示', '支持', '女性', '龍鳳胎', '故事']
#那麼我們把這個List裡面的所有元素依序丟入model裡面，每個元素會被換算出一組300維的向量。
#那個(300,0)x15的向量該怎麼儲存比較好呢？List嗎?這樣就會是一個由15組array為元素的List，是這樣嗎??
#對了，某些專有名詞或人名沒有被訓練在word vector裡面...所以...要怎麼辦呢?
#某些討論有提到，(1)未知詞使用『模型平均向量』填充  (2)未知詞使用『全零向量』填充
#假設是這樣的話就這樣寫:

ldavec_list = [] #由15組詞向量作為元素的List
unknown_list = [] #把未知詞都另外添加到其他List，事後回來檢查有沒有重要詞是未知詞

for i in lda_keywords[0]:
    if i in model.keys(): #檢查詞有沒有被訓練在模型裡面，沒有就是未知詞
        ldavec_list.append(model[i])
    else:
        ldavec_list.append(unknown)
        unknown_list.append(i)
print(len(ldavec_list)) #檢查由15組詞向量作為元素的List的長度
print(ldavec_list[0]) #檢查第1個向量
print(unknown_list) #檢查未知詞名單